<a href="https://colab.research.google.com/github/s-jainr/deep-learning-sp23/blob/main/Deep_Learning_Final_Project_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytreebank
import pytreebank
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset from disk
dataset = pytreebank.load_sst("/path/to/sentiment/")

# Extract the train, validation, and test data
train_data = dataset['train'][:1000]
# validation_data = dataset['dev']
# test_data = dataset['test']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Build the vocabulary
vocab = set()
for sentence in train_data:
    sentence = sentence.to_labeled_lines()[0][1]
    words = sentence.lower().split()
    for word in words:
        vocab.add(word)

# Create the word-to-index and index-to-word mappings
word_to_index = {}
index_to_word = {}
for i, word in enumerate(vocab):
    word_to_index[word.lower()] = i
    index_to_word[i] = word.lower()

# Convert the sentences to sequences of word indices
train_sequences = []
for sentence in train_data:
    sentence = sentence.to_labeled_lines()[0][1]
    words = sentence.lower().split()
    sequence = [word_to_index[word.lower()] for word in words]
    train_sequences.append(sequence)

# Pad the sequences to a fixed length
max_length = max(len(sequence) for sequence in train_sequences)
train_sequences = np.array([sequence + [0]*(max_length-len(sequence)) for sequence in train_sequences])

# Convert the labels to one-hot vectors
train_labels = np.zeros((len(train_data), 5))
for i, sentence in enumerate(train_data):
    train_labels[i][sentence.label] = 1

In [3]:
train_x = []
for seq in train_sequences:
  matrix = []
  for encoding in seq:
    seq_hot = np.zeros(len(vocab))
    seq_hot[encoding] = 1
    matrix.append(seq_hot)
  train_x.append(matrix)

train_x = np.array(train_x)

In [4]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_labels, test_size=0.2, random_state=47)

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=(52, 4806)))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 50, 32)            461408    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 25, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 23, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 11, 64)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 9, 128)            24704     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 4, 128)           0

In [8]:
es = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

network_history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=1000, batch_size=32, callbacks = [es])

Epoch 1/1000
25/25 [==============================] - 1s 52ms/step - loss: 1.1347 - accuracy: 0.4913 - val_loss: 1.2310 - val_accuracy: 0.4750
Epoch 2/1000
25/25 [==============================] - 1s 30ms/step - loss: 1.0807 - accuracy: 0.4913 - val_loss: 1.2210 - val_accuracy: 0.4750
Epoch 3/1000
25/25 [==============================] - 1s 41ms/step - loss: 0.9196 - accuracy: 0.5025 - val_loss: 1.2726 - val_accuracy: 0.4550
Epoch 4/1000
25/25 [==============================] - 1s 47ms/step - loss: 0.5449 - accuracy: 0.7912 - val_loss: 2.2213 - val_accuracy: 0.3000
Epoch 5/1000
25/25 [==============================] - 1s 43ms/step - loss: 0.2766 - accuracy: 0.8925 - val_loss: 3.2573 - val_accuracy: 0.3300
Epoch 6/1000
25/25 [==============================] - 1s 46ms/step - loss: 0.1418 - accuracy: 0.9538 - val_loss: 3.5782 - val_accuracy: 0.3500


In [37]:
correct, incorrect, accuracy = 0, 0, 1

for i in range(len(test_x)):
  sequence = test_x[i]
  true_label = test_y[i]

  prediction = model.predict(np.reshape(sequence, newshape=(1, sequence.shape[0], sequence.shape[1])), verbose=0)

  if np.argmax(prediction) == np.argmax(true_label):
    correct += 1
  else:
    incorrect += 1
  
  accuracy = correct / (correct + incorrect)

print(f"accuracy: {accuracy:.2f}")
  

accuracy: 0.47
